Below is setup for Drake

In [0]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib.
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

In [0]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import sys 
import platform
import pydrake
from pydrake.solvers import branch_and_bound

# pydrake imports
from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, LinearQuadraticRegulator,
                         Linearize, AddMultibodyPlantSceneGraph, VectorSystem, le, ge, OsqpSolver, PrismaticJoint,
                         Cylinder,Box,RigidTransform, CoulombFriction, GetInfeasibleConstraints)
# underactuated imports
from underactuated import FindResource

In [183]:
#clears file and downloads urdf to have it be updated
get_ipython().system(u"rm -rf /opt/hopper")
if platform.system() == "Darwin":
  get_ipython().system(u"if [ ! -d '/opt/hopper' ]; then git clone https://github.com/Olliebear/underactuated_final_project.git /opt/hopper; fi")
elif platform.linux_distribution() == ("Ubuntu", "18.04", "bionic"):
  get_ipython().system(u"if [ ! -d '/opt/hopper' ]; then git clone https://github.com/Olliebear/underactuated_final_project.git /opt/hopper; fi" )
else:
  assert False, "Unsupported platform"
sys.path.append("/opt/hopper")

Cloning into '/opt/hopper'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 105 (delta 63), reused 73 (delta 31), pack-reused 0
Receiving objects: 100% (105/105), 2.93 MiB | 22.52 MiB/s, done.
Resolving deltas: 100% (63/63), done.


##Define the problem's parameters:
x0, y0 are coordinates of the foot of the 2d hopper.

th1 (theta1) is the angle between the foot and the upright, th2 is angle between bode and upright.

z0 is the distance from the hip pin joint to the foot. 

uT is torque input at pinjoint connecting leg and body. uX is length actuation on the spring in the leg. 

###Other parameters:
Spring length K_length= z0 - uX.

m1,I1,r1 = mass, inertia and radius from toe to C.O.M. for the leg

m2,I2,r2 = mass, inertia and radius from hip to C.O.M. for the body

##While on the ground:
For now, assume infinite friction. Can add friction cone constraint later, or add more modes

###Define Some System Constants
Order of states: x0, y0, th1, z0, th2

# **Some info here**
In general, this code is a bit of a crap heap. Sorry. Lots of things/ideas going on here that I'm working on. The next few blocks though are the main part I think.

The problem is I need a better solver to actually solve my mathematical program, but I'm not sure how to get that onto this colab machine remotely. I'd imagine there's licensing issues. 

Also, it would be difficult for me to install drake on my personal computer because it would involve updating my OS (to get python 3.8)

In [184]:

builder=DiagramBuilder()
hopper, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
Parser(hopper).AddModelFromFile('/opt/hopper/hopper.urdf')
#leg_col_shape=Cylinder(0.05,0.2)
#ground_col_shpae=Box(100,1,2)
#leg=hopper.GetBodyByName('leg')
#ground=hopper.GetBodyByName('ground')
#leg_t=RigidTransform()
#ground_t=RigidTransform()

#friction=CoulombFriction(0.6,0.5) #static, dynamic

#hopper.RegisterCollisionGeometry(leg,leg_t,leg_col_shape,"leg",friction)
#print(hopper.GetCollisionGeometriesForBody(ground))
hopper.Finalize()
hopper=hopper.ToAutoDiffXd()
#hopper.set_penetration_allowance(0.1)
#print(hopper.get_contact_penalty_method_time_scale())
#a=hopper.GetJointByName("z0")#
#print(a)
#print(a.damping())

nq=hopper.num_positions()
print(nq)

nf=4 # 2 inputs ux ut and 2 contact forces

context=hopper.CreateDefaultContext()
context.SetTime(0)


5


In [0]:
def make_animation(x_trj,h_trj,vis_hopper):

  builder=DiagramBuilder()
  scene_graph = SceneGraph()
  builder.AddSystem(scene_graph)
  vis_hopper.RegisterAsSourceForSceneGraph(scene_graph)

  Parser(visual_hopper).AddModelFromFile('/opt/hopper/hopper.urdf')
  visual_hopper.Finalize()
  #pretty much all of this was copied from compass gait problem in the homework
  time_breaks_opt = np.array([sum(h_trj[:t]) for t in range(len(h_trj)+1)])
  x_trj_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_trj.T)

  source = builder.AddSystem(TrajectorySource(x_trj_poly))
  pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(vis_hopper))
  builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())

  builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(vis_hopper.get_source_id()))

  xlim = [-2, 2.]
  ylim = [-.2, 3]
  visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
  builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
  simulator = Simulator(builder.Build())

  # generate and display animation
  visualizer.start_recording()
  simulator.AdvanceTo(x_trj_poly.end_time())
  ani = visualizer.get_recording_as_animation()
  return ani

In [0]:
def manipulator_equations(vars):
    
    # split input vector in subvariables
    # configuration, velocity, acceleration, stance-foot force
    assert vars.size == 3 * nq + nf
    split_at = [nq, 2 * nq, 3 * nq , 3*nq+2]
    q, qd, qdd, u,f = np.split(vars, split_at)
    spring_f=get_spring_force(q[3],u[0])
    forces=[spring_f,u[1]]
    # set compass gait state
    context = hopper.CreateDefaultContext()
    hopper.SetPositions(context, q)
    hopper.SetVelocities(context, qd)
    
    # matrices for the manipulator equations
    M = hopper.CalcMassMatrixViaInverseDynamics(context)
    Cv = hopper.CalcBiasTerm(context)
    tauG = hopper.CalcGravityGeneralizedForces(context)
    #changing to makeActuation_matrix() then multiplying that by u.
    B=hopper.MakeActuationMatrix()
    # Jacobian of the stance foot
    Jf = get_foot_jacobian(hopper, context)

    # return violation of the manipulator equations
    return M.dot(qdd) + Cv - tauG  - B.dot(forces) - Jf.T.dot(f)

In [0]:
def get_foot_jacobian(hopper, context):
    
    # get reference frames for the given leg and the ground
    leg_frame = hopper.GetBodyByName('leg').body_frame()
    ground_frame = hopper.GetBodyByName('ground').body_frame()

    # compute Jacobian matrix
    J = hopper.CalcJacobianTranslationalVelocity(
        context,
        JacobianWrtVariable(0),
        leg_frame,
        np.array([0,0,0]),
        ground_frame,
        ground_frame
    )
    
    # discard y components since we are in 2D
    return J[[0, 2]]

In [0]:
def manipulator_equations_2(vars):
  # split input vector in subvariables
    # configuration, velocity, acceleration, stance-foot force
    assert vars.size == 3 * nq + 3
    split_at = [nq, 2 * nq, 3 * nq ,3*nq+2]
    q, qd, qdd, u,Fx = np.split(vars, split_at)

    r1,r2,m1,m2,I1,I2=[0.5,0.7,1,10,1,10]
    
    Fy=get_ground_spring(q[1],qd[1])
    #Fx=get_ground_friction(fy)
    
    g=9.81
    #F_spring=u[0] #make spring force decision variable, not length.
    c1=np.cos(q[2])
    c2=np.cos(q[4])
    s1=np.sin(q[2])
    s2=np.sin(q[4])
    c12=np.cos(q[4]-q[2])
    s12=np.sin(q[4]-q[2])
    
    
    z=q[3]+1
    W=z-r1
    uT=u[1]

    F_spring=get_spring_force(z,u[0])


    dth1=qd[2]
    dth2=qd[4]
    dz=qd[3]
    ddth1=qdd[2]
    ddth2=qdd[4]
    ddx=qdd[0]
    ddy=qdd[1]
    ddz=qdd[3]

    eq1=c1*(m2*W*z + I1)*ddth1 + m2*r2*W*c2*ddth2 + m2*W*ddx + m2*W*s1*ddz - (W*m2*(dth1**2*W*s1 - 2*dth1*dz*c1+r2*dth2**2*s2+r1*dth1**2*s1) -r1*Fx*c1**2 + c1*(r1*Fy*s1-uT) + F_spring*W*s1)
    eq2=-s1*(m2*W*z + I1)*ddth1 - m2*r2*W*s2*ddth2 + m2*W*ddy + m2*W*c1*ddz - (W*m2*(dth1**2*W*c1 + 2*dth1*dz*s1+r2*dth2**2*c2+r1*dth1**2*c1 - g) + r1*Fx*c1*s1 - s1*(r1*Fy*s1-uT) + F_spring*W*c1)
    eq3=c1*(m1*r1*W-I1)*ddth1 + m1*W*ddx - (W*(m1*r1*dth1**2*s1 - F_spring*s1 +Fx) -c1*(Fy*r1*s1 - Fx*r1*c1 - uT))
    eq4=-s1*(m1*r1*W-I1)*ddth1 + m1*W*ddy - (W*(m1*r1*dth1**2*c1 - F_spring*c1 +Fy - m1*g) - s1*(Fy*r1*s1 - Fx*r1*c1 - uT))
    eq5=-c12*I1*r2*ddth1 + I2*z*ddth2 - (W*(F_spring*r2*s12+uT) -r2*c12*(r1*Fy*s1-r1*Fx*c1-uT))
    
    # alternate to make things easier?
    #eq2= m2*W*ddy + m2*W*ddz - (W*m2*( - g) + F_spring*W)
    #eq4= m1*W*ddy - (W*( - F_spring +Fy - m1*g))


    # return violation of the manipulator equations
    return [eq1,eq2,eq3,eq4,eq5]

In [0]:
def get_spring_force(z0,uX):
  #k=1000 #spring constant
  #kL=1 #relaxed length
  #if z0>0:
    #k=k*100
    #uX=0
  #return k*(-z0+uX)

  #this is if uX is stiffness variable
  #if z0>0: #if above 0, backwards force is stronger
    #uX=uX*100
  #return -uX*z0

  k=2500
  k0=1
  if (k0-z0+uX)>0:
    return k*(k0-z0+uX)
  else:
    return k*100*(k0-z0+uX)

In [0]:
def get_ground_height(x):
  #use this to get the height at some position x. can change to have variable terrain
  return 0

In [0]:
def get_ground_schedule(num_steps,num_hits,density=False):
  schedule=np.zeros(num_steps)
  if density:
    schedule=None
  else:#first time running, no prescribed density
    total_num=num_hits*2+1 #starts in air, ends on ground, 
    steps_per_hit=int(num_steps/total_num)
    for i in range(total_num):
      if (i%2==1): #if odd, on the ground
        lower=int((i+1/4)*steps_per_hit)
        upper=int((i+3/4)*steps_per_hit)
        schedule[lower:upper]=np.ones(upper-lower)
  return schedule

In [192]:
#FIND STEADY STATE SOLUTION AT x=0 with hopping height y_d
prog=MathematicalProgram()
N=200

y_d = 0.5 #desired hopping height

q_initial=[0,y_d,0,0,0]
qd_initial=[0,0,0,0,0]

h_max=0.1
h_min=.001

context = hopper.CreateDefaultContext()
hopper.SetPositions(context, q_initial)
hopper.SetVelocities(context, qd_initial)

h=prog.NewContinuousVariables(N,1,'h') #time steps
q=prog.NewContinuousVariables(N+1,5,'q')
qd=prog.NewContinuousVariables(N+1,5,'qd')
qdd=prog.NewContinuousVariables(N,5,'qdd')
uZ=prog.NewContinuousVariables(N,1,'uZ')
uT=prog.NewContinuousVariables(N,1,'uT')
f_ground=prog.NewContinuousVariables(N,2,'f_ground')

#make grounded variable pre-scheduled 
number_of_contacts=1
grounded=get_ground_schedule(N,number_of_contacts)
M=100
Mf=10000
#print(state[0])

prog.AddLinearConstraint(eq(q[0],q_initial))
prog.AddLinearConstraint(eq(qd[0],qd_initial))

#prog.AddLinearConstraint(eq(q[N],[0,0.2,0,0,0]))
#prog.AddLinearConstraint(eq(qd[N],[0,0,0,0,0]))

prog.AddBoundingBoxConstraint(np.zeros(len(uZ)),np.ones(len(uZ))*0.8,uZ)

prog.AddBoundingBoxConstraint(h_min*np.ones(len(h)),h_max*np.ones(len(h)),h)



for t in range(N):

  prog.AddConstraint(eq(q[t+1] , q[t] + h[t] * qd[t+1]))
  prog.AddConstraint(eq(qd[t+1] , qd[t] + h[t] * qdd[t]))


  var=np.concatenate((q[t],qd[t],qdd[t],uZ[t],uT[t],f_ground[t]))
  prog.AddConstraint(manipulator_equations, lb=[0]*nq, ub=[0]*nq, vars=var) #dynamics constraint
  #prog.AddLinearConstraint(le(uZ[t],0))
  #y force is > 0
  prog.AddLinearConstraint(f_ground[t,1]>=0)

  prog.AddLinearConstraint(f_ground[t,1]<=grounded[t]*Mf)

  prog.AddLinearConstraint(f_ground[t,0]==0)
  #y0 is always above ground. using get_ground_height so that we can change ground geometry
  #prog.AddConstraint(foot_height,lb=[0],ub=[(1-grounded[t])*M],vars=q[t])

  #below is complimentarity constraint
  #prog.AddLinearConstraint(q[t,3]<=0)
  #if touching ground, xdot = 0, ydot=0, y=0

  prog.AddLinearConstraint(q[t+1,1]<=(1-grounded[t])*M) #complimentarity
  prog.AddLinearConstraint(q[t+1,1]>=-.1) #y is above 0 , -.1 for sanity
  #prog.AddLinearConstraint(ge(q[t,3],-1*grounded*M))
  #prog.AddLinearConstraint(q[t,3]>=0)
  #prog.AddLinearConstraint
  #add controller cost
  #prog.AddCost(uZ[t].dot(uZ[t]))
  prog.AddLinearConstraint(eq(uZ[t],0))

#q_guess=q_star
#q_guess[:,3]=np.array(get_ground_schedule(N+1,number_of_contacts))*-.3
#prog.SetInitialGuess(q,q_guess)


solver=SnoptSolver()
result=solver.Solve(prog)
print(result.is_success())
h_star=result.GetSolution(h)
q_star=result.GetSolution(q)
qd_star=result.GetSolution(qd)
qdd_star=result.GetSolution(qdd)
steps=np.linspace(0,N,N+1)
plt.figure()
plt.plot(steps,q_star[:,1])
uZ_star=result.GetSolution(uZ)
#print(uZ_star)
#print(q_star[:,1])
plt.plot(steps,q_star[:,3])
#plt.plot(steps[1:],uZ_star)
#plt.plot(steps[1:],qdd_star[:,1])
plt.figure()
plt.plot(steps[1:],result.GetSolution(f_ground)[:,1])

KeyboardInterrupt: ignored

In [43]:

visual_hopper =MultibodyPlant(time_step=0)
a=make_animation(q_star,h_star,visual_hopper)
HTML(a.to_jshtml())

In [0]:
def get_ground_spring(y,yd):
  kg=10000
  cv=0 #damping
  return -kg*y 

In [0]:
def spring_constraint(var):
  k=1000
  z=var[0]
  uZ=var[1]

  return [abs(uZ)-abs(k*z)]

In [0]:
def get_guess(initial,final,steps,min_dt,max_dt):
  dt=(min_dt+max_dt)/2
  y=np.ones(steps)*initial[1]
  x=np.ones(steps)*initial[0]
  start_touch=None
  vy=-9.8*dt
  vx=0
  z=np.ones(steps)*initial[3]
  max_ground=.3 #max time on ground

  for step in range(steps-1):
    if y[step]>0: #in air
      vy=vy-9.8*dt
      y_next=y[step]+vy*dt
      vx=0
      z_next=0
    if y[step]<=0: #on ground
      if not start_touch:
        start_touch=step
      y_next=0

      num=np.ceil(max_ground/dt)
      z_next=get_quadratic(num,step-start_touch)
      if (step-start_touch)*dt>max_ground: #ready to move off ground
        vy=abs(vy)*.75
        y_next=0.01
        vx=.2*(final[0]-initial[0]) #how much you go over
    y[step+1]=y_next
    x[step+1]=x[step]+vx*dt
    z[step+1]=z_next
  q=np.ones((steps,5))
  for step in range(steps):
    q[step]=np.array([ x[step],y[step],0,z[step],0 ])
  return q

def get_quadratic(num_steps,step):
  max_val=-0.6
  min_val=0
  x=step
  return -(x)*(x-num_steps)*4*max_val/(num_steps**2) 

In [1]:
#try it without multibody plant and instead use dynamics equations
prog=MathematicalProgram()
N=100

y_d = 0.5 #desired hopping height

q_initial=[0,y_d,0,0,0]
qd_initial=[0,0,0,0,0]

q_final=[0,y_d/2,0,0,0]

h_max=.01
h_min=.005

h=prog.NewContinuousVariables(N,1,'h') #time steps
q=prog.NewContinuousVariables(N+1,5,'q')
qd=prog.NewContinuousVariables(N+1,5,'qd')
qdd=prog.NewContinuousVariables(N,5,'qdd')
uZ=prog.NewContinuousVariables(N,1,'uZ') #now make it force not length
uT=prog.NewContinuousVariables(N,1,'uT')
#lambdas=prog.NewContinuousVariables(N,2,'lambdas')
Fx=prog.NewContinuousVariables(N,1,'Fx')
#print(q)
q_guess=get_guess(q_initial,q_initial,N+1,h_min,h_max)
#print(q_guess)
#prog.SetInitialGuess(q,q_guess)
#prog.SetInitialGuess(qd,qd_star)
#prog.SetInitialGuess(qdd,get_guess(N,5))
#prog.SetInitialGuess(uT,get_guess(N,1))
#prog.SetInitialGuess(uZ,get_guess(N,1))


#make grounded variable pre-scheduled 
number_of_contacts=2
grounded=get_ground_schedule(N,number_of_contacts)
M=100
Mf=10000
#print(state[0])

prog.AddLinearConstraint(eq(q[0],q_initial))
prog.AddLinearConstraint(eq(qd[0],qd_initial))

#prog.AddLinearConstraint(eq(q[N],q_final))
#prog.AddLinearConstraint(eq(qd[N],qd[0]))
#prog.AddCost((q[N]-q_final).dot(q[N]-q_final))

#prog.AddBoundingBoxConstraint([1000]*N,[3000]*N,uZ) #for uZ as a stiffness modifier for the spring
prog.AddBoundingBoxConstraint([0]*N,[0.8]*N,uZ)

qdd_initial=[0,-9.8,0,0,0]

prog.AddLinearConstraint(eq(qdd[0],qdd_initial))

k=1000 #change in spring_constraint too
for t in range(N):
  prog.AddBoundingBoxConstraint(-1*np.ones(1),0.05*np.ones(1),q[t+1,3])
  #vals=np.array([q[t,3],uZ[t,0]])
  #prog.AddConstraint(spring_constraint, [0], [k], vars=vals) #constraint for force uZ. 
  #prog.AddConstraint(le(uZ[t].dot(q[t,3]),0))
  #prog.AddLinearConstraint(le(uZ[t],-k*(q[3]+0.8)))

  prog.AddBoundingBoxConstraint(h_min*np.ones(1),h_max*np.ones(1),h[t])

  prog.AddConstraint(eq(q[t+1] , q[t] + h[t] * qd[t+1]))
  prog.AddConstraint(eq(qd[t+1] , qd[t] + h[t] * qdd[t]))

  var=np.concatenate((q[t+1],qd[t+1],qdd[t],uZ[t],uT[t],Fx[t]))
  prog.AddConstraint(manipulator_equations_2, lb=[0]*nq, ub=[0]*nq, vars=var) #dynamics constraint
  
  #y force is > 0

  #y0 is always above ground. using get_ground_height so that we can change ground geometry
  #prog.AddConstraint(foot_height,lb=[0],ub=[(1-grounded[t])*M],vars=q[t])

  #below is complimentarity constraint
  #prog.AddLinearConstraint(q[t,3]>=-0.8)
  #if touching ground, xdot = 0, ydot=0, y=0

  #prog.AddLinearConstraint(qd[t,0]<=(1-grounded[t])*M)
  #prog.AddLinearConstraint(qd[t,0]>=-1*(1-grounded[t])*M)
  #prog.AddLinearConstraint(qd[t,1]<=(1-grounded[t])*M)
  #prog.AddLinearConstraint(qd[t,1]>=-1*(1-grounded[t])*M)

  #prog.AddLinearConstraint(lambdas[t,0]>=-1*(1-grounded[t])*Mf)
  #prog.AddLinearConstraint(lambdas[t,0]<=(1-grounded[t])*Mf)
  #prog.AddLinearConstraint(lambdas[t,1]<=(grounded[t])*Mf)
  #prog.AddLinearConstraint(lambdas[t,1]>=0)
  
  prog.AddLinearConstraint(le(Fx[t],1.5*get_ground_spring(q[t,0],q[t,1])))
  prog.AddLinearConstraint(ge(Fx[t],-1.5*get_ground_spring(q[t,0],q[t,1])))

  
  #prog.AddLinearConstraint(le(uZ[t],(grounded[t])*M))


  #prog.AddLinearConstraint([])
  #prog.AddLinearConstraint()
  #prog.AddLinearConstraint(q[t,1]-get_ground_height(q[t,0])<=(1-grounded[t])*M) #complimentarity
  #prog.AddLinearConstraint(q[t,1]-get_ground_height(q[t,0])>=0) #y is above 0
  #add controller cost
  #prog.AddCost(q[t,1]**2)
  prog.AddLinearConstraint(eq(uZ[t],0))

solver=SnoptSolver()
result=solver.Solve(prog)
print(result.is_success())
h_star=result.GetSolution(h)
q_star=result.GetSolution(q)
qd_star=result.GetSolution(qd)
steps=np.linspace(0,N,N+1)
plt.figure()
plt.plot(steps,q_star[:,1])
uZ_star=result.GetSolution(uZ)
print(grounded)
#print(uZ_star)
#print(q_star[:,1])
plt.plot(steps,q_guess[:,1])
plt.figure()
plt.plot(steps,q_star[:,3])
plt.plot(steps,q_guess[:,3])
plt.figure()
plt.plot(steps[1:],uZ_star)
print(GetInfeasibleConstraints(prog,result))
print(qd_star[0,1],qd_star[1,1])
print(qd_star[:,1])

NameError: ignored

In [291]:
print(h_star)
visual_hopper =MultibodyPlant(time_step=0)
a=make_animation(q_star,h_star,visual_hopper)
HTML(a.to_jshtml())


[0.01       0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.005      0.005      0.005      0.005      0.005      0.005
 0.01       0.005      0.00527677 0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.00587505 0.005
 0.005      0.005      0.005      0.005      0.005      0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.00755217 0.01       0.005      0.005      0.005      0.005
 0.005      0.005      0.0065805  0.01       0.01       0.005
 0.005      0